In [1]:
using PowerSystems
using BenchmarkTools
cd(string(homedir(),"/.julia/v0.6/PowerSystems/data"))
include("data_5bus.jl");

In [2]:
include("data_5bus.jl");

In [3]:
using JuMP
using MathOptInterface
const MOI = MathOptInterface
const MOIU = MOI.Utilities
using Gurobi
solver = GurobiSolver()

Gurobi.GurobiSolver(nothing, Any[])

In [6]:
m = Model()   
@variable(m, x[["a","b"]])

1-dimensional JuMPArray{JuMP.VariableRef,1,...} with index sets:
    Dimension 1, String["a", "b"]
And data, a 2-element Array{JuMP.VariableRef,1}:
 x[a]
 x[b]

In [9]:
x.axes[1]

2-element Array{String,1}:
 "a"
 "b"

In [ ]:
function GenerationVariables(m::JuMP.Model, generators::Array{T,1}, T) where T <: Thermal 
    g_on_set = [g.name for g in generators if g.available == true]
    t = 1:T
    @variable(m::JuMP.Model, P_g[g_on_set,t]) # Power output of generators
    return true    
end

In [ ]:
function BranchFlowVariables(m::JuMP.Model, PowerSystem::PowerSystem)
    br_on_set = [br.name for br in PowerSystem.network.branches if br.available == true]
    t = 1:PowerSystem.timesteps
    @variable(m, fbr[br_on_set,t])
    return true   
end

In [ ]:
function ControlableLoadVariables(m::JuMP.Model, PowerSystem::PowerSystem)
    cl_on_set = [cl.name for cl in PowerSystem.loads if (cl.available == true && !isa(cl,StaticLoad))]
    t = 1:PowerSystem.timesteps
    @variable(m, pcl[cl_on_set,t]) 
    return true              
end

In [ ]:
function StorageVariables(m::JuMP.Model, PowerSystem::PowerSystem)
    cl_on_set = [cl.name for cl in PowerSystem.loads if (cl.available == true && typeof(cl) != StaticLoad)]
    t = 1:PowerSystem.timesteps
    @variable(m, pcl[cl_on_set,t]) 
    return true              
end

In [ ]:
sys5 = PowerSystem(nodes5, generators5, loads5_DA, branches5, 230.0, 1000.0);

In [ ]:
sys5.generators

In [ ]:
function GenerationVariables(m::JuMP.Model, generators::Array{D,1}, T::Int) where D <: Generator 
    g_on_set = [g.name for g in generators if g.available == true]
    t = 1:T
    @variable(m::JuMP.Model, pth[g_on_set,t]) # Power output of generators
    return true    
end

In [ ]:
function PowerLimitConstraints(m::JuMP.Model, generators::Array{D,1}) where D <: Generator
    sets = m[:pth].indexsets
    
    @constraintref Pmax[sets]
    
end    

In [ ]:
m=Model()
gens = sys5.generators["Thermal"];

In [ ]:
GenerationVariables(m, gens, 24)

In [ ]:
sets = m[:pth].indexsets
@constraintref Pmax[]

In [ ]:
PowerLimitConstraints(m, gens)

In [ ]:
@variable(m, x[["a", "b"]])

In [ ]:
s = x.indexsets

In [ ]:
@macroexpand @constraintref foo10[1:length(s[1]), 1:length(s[2])]

In [ ]:
typeof([1:10])